# Records

::: {.content-hidden when-format="html"}

In [7]:
#| echo: false
#| output: false
import os
os.environ['R_HOME'] = f'C:/Users/{os.environ.get('USERNAME')}/Miniconda3/envs/r_python_jl/Lib/R'

In [8]:
#| echo: false
#| output: false
# enables the %%R magic, not necessary if you've already done this
%load_ext rpy2.ipython
# only have to run once to allow the R magic command

C:\Users\ekp0303\Miniconda3\envs\r_python_jl\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


::: {.panel-tabset}

#### R

In [9]:
%%capture --no-display 
%%R
library("dplyr")
library("jsonlite")
library("tidyr")
library("REDCapR")
library("knitr")
library("remotes")
library("gt")

In [10]:
%%capture --no-display --no-stdout
%%R

# Detach REDCapR if already loaded, and download the latest version
if (version!='1.1.9005') {
    detach("package:REDCapR", unload=TRUE)
    remotes::install_github("OuhscBbmc/REDCapR")
    library("REDCapR")
    print(packageVersion("REDCapR"))
}

else {
    print("REDCapR package up to date")
}

[1] '1.1.9005'


In [11]:
%%R
# Load API tokens from the json file
token <- jsonlite::fromJSON('./../../json_api_data.json')$dev_token$'309'
url <- "https://dev-redcap.doh.wa.gov/api/"

#### Python

In [12]:
import redcap
import json
import csv
import pandas as pd
import numpy as np
import requests
import tempfile

In [13]:
# Load API tokens from the json file
key = json.load(open('./../../json_api_data.json'))
token = key['dev_token']['309']
url = key['dev_url']

project = redcap.Project(url, token)

:::

:::

::: {.panel-tabset}

#### R


`redcap_write_oneshot()` and `redcap_write()`

Records can be imported into a REDCap project from a dataframe in R using `redcap_write_oneshot()` to write a records all at once, or using `redcap_write()` which can batch the records to be imported so the server is not overwhelmed in the case of large imports. These methods will accept either an R dataframe or tibble containing the data to be imported. 

If the `record_id`(s) being imported already exists in the REDCap project, the imported data will overwrite the previously existing data for that record. Using a `record_id` that does not already exist will create a new record. See @sec-next_record on how to use the API to find the next available `record_id`.

The `overwrite_with_blanks` argument is set to 'FALSE' by default; under this setting, if blank values are imported for fields on existing REDCap records and that data is not missing in REDCap, these values will not be overwritten as missing. If you want to overwrite existing data as missing, be sure to use `overwrite_with_blanks = 'TRUE'.`

In [14]:
%%capture --no-display 
%%R
# Define data to import
df1 <- data.frame(record_id = c(7,8),
                  first_name = c("John","Jane"),
                  last_name = c("Doe","Doe")
                  )

In [15]:
%%capture --no-stdout 
%%R
redcap_write_oneshot(df1, redcap_uri=url, token=token)

$success
[1] TRUE

$status_code
[1] 200

$outcome_message
[1] "2 records were written to REDCap in 0.6 seconds."

$records_affected_count
[1] 2

$affected_ids
[1] "7" "8"

$elapsed_seconds
[1] 0.634845

$raw_text
[1] ""



In [16]:
%%R
df2 <- data.frame(record_id = 9,
                  first_name = "John",
                  last_name = "Doe"
                  )

In [17]:
%%capture --no-stdout 
%%R
redcap_write(df2, redcap_uri=url, token=token)
#optional argument: batch_size = 100 (default)

$success
[1] TRUE

$status_code
[1] "200"

$outcome_message
[1] "1 records were written to REDCap in 0.4 seconds."

$records_affected_count
[1] 1

$affected_ids
[1] "9"

$elapsed_seconds
[1] 0.934711



#### Python

`import_records()`

Data can be imported as a pandas dataframe, json, csv, or xml, specified by the `import_format` argument (default is json).

If the `record_id`(s) being imported already exists in the REDCap project, the imported data will overwrite the previously existing data for that record. Using a `record_id` that does not already exist will create a new record. The `force_auto_number = 'True'` argument will automatically reassign existing `record_ids` to new `record_ids` during import. If set to 'False' and your Record ID's to import already exist in REDCap, they will overwrite the existing REDCap records during import. You can also see @sec-next_record on how to use the API to find the next available `record_id`.

The `overwrite` argument is set to 'normal' by default; under this setting, if blank values are imported for fields on existing REDCap records and that data is not missing in REDCap, these values will not be overwritten as missing. If you want to overwrite existing data as missing, be sure to use `overwrite = 'overwrite'.`

In [25]:
df_py = [{'record_id': 7,
  'redcap_event_name': 'personal_info_arm_1',
  'redcap_repeat_instrument': '',
  'redcap_repeat_instance': None,
  'first_name': 'John',
  'last_name': 'Doe'},
 {'record_id': 8,
  'redcap_event_name': 'personal_info_arm_1',
  'redcap_repeat_instrument': '',
  'redcap_repeat_instance': None,
  'first_name': 'Jane',
  'last_name': 'Doe'}]

project.import_records(df_py, force_auto_number=True)

{'count': 2}

:::

**Note:** For troubleshooting import errors, please thoroughly review @sec-import_appendix. This chapter goes into to detail about the limitations to importing and provides more detailed import examples. 